# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [523]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [524]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [525]:
train_data,test_data = sales.random_split(.8,seed=0)

In [526]:
import math
from math import log
#[log(y,10) for y in x]

In [527]:
train_data['bedrooms_squared'] = train_data['bedrooms']*train_data['bedrooms']
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
train_data['log_sqft_living'] = [math.log(x,10) for x in train_data['sqft_living']]

In [528]:
train_data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [529]:
test_data['bedrooms_squared'] = test_data['bedrooms']*test_data['bedrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']
test_data['log_sqft_living'] = [math.log(float(x),) for x in test_data['sqft_living']]

In [530]:
test_data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980,2,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553,2,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186,1,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495,2,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400,1,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807,1.5,0


In [531]:
bed_mean = test_data['bedrooms_squared'].mean()
bed_bath_mean = test_data['bed_bath_rooms'].mean()
lat_long_mean = test_data['lat_plus_long'].mean()
log_mean = test_data['log_sqft_living'].mean()

In [532]:
print bed_mean
print bed_bath_mean
print lat_long_mean
print log_mean

12.4466777016
7.50390163159
-74.6533349722
7.55027467965


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [533]:
model1 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model2 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']
model3 = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 
          'bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
model1_reg = graphlab.linear_regression.create(train_data, target = 'price', features = model1, 
                                               validation_set = None, verbose=False)

model2_reg = graphlab.linear_regression.create(train_data, target = 'price', features = model2, 
                                               validation_set = None, verbose=False)

model3_reg = graphlab.linear_regression.create(train_data, target = 'price', features = model3, 
                                               validation_set = None, verbose=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [534]:
model1_weight_summary = model1_reg.get("coefficients")
model2_weight_summary = model2_reg.get("coefficients")
model3_weight_summary = model3_reg.get("coefficients")

print model1_weight_summary
print model2_weight_summary
print model3_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56140675.7444 | 1649985.42028 |
| sqft_living |  None | 310.263325778  | 3.18882960408 |
|   bedrooms  |  None | -59577.1160682 | 2487.27977322 |
|  bathrooms  |  None | 13811.8405418  | 3593.54213297 |
|     lat     |  None | 629865.789485  | 13120.7100323 |
|     long    |  None | -214790.285186 | 13284.2851607 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]

+----------------+-------+----------------+---------------+
|      name      | index |     value      |     stderr    |
+----------------+-------+----------------+---------------+
|  (intercept)   |  None | -54410676.1152 | 1650405.16541 |
|  sqft_living   |  None | 304.449298057  | 3.20217535637 |
|    bedrooms    |  None | -116366.043231 | 4805.54966546 |
|   bathrooms    |  None | -77972.3305135 | 7565

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [535]:
model1_predictions = model1_reg.predict(train_data)
print model1_predictions[0] # should be 271789.505878
model2_predictions = model2_reg.predict(train_data)
print model2_predictions[0]
model3_predictions = model3_reg.predict(train_data)
print model3_predictions[0]

245784.01808
252842.847289
285113.12819


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [536]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predicted = model.predict(data)
    # Then compute the residuals/errors
    residuals = predicted - outcome
    # Then square and add them up
    RSS = residuals*residuals
    RSS = RSS.sum()
    return(RSS)     

In [537]:
rss_model1_train = get_residual_sum_of_squares(model1_reg, train_data, train_data['price'])
print rss_model1_train
rss_model2_train = get_residual_sum_of_squares(model2_reg, train_data, train_data['price'])
print rss_model2_train
rss_model3_train = get_residual_sum_of_squares(model3_reg, train_data, train_data['price'])
print rss_model3_train

9.71328233544e+14
9.61592067856e+14
9.05276314555e+14


Test your function by computing the RSS on TEST data for the example model:

In [538]:
rss_model1_test = get_residual_sum_of_squares(model1_reg, test_data, test_data['price'])
print rss_model1_test
rss_model2_test = get_residual_sum_of_squares(model2_reg, test_data, test_data['price'])
print rss_model2_test
rss_model3_test = get_residual_sum_of_squares(model3_reg, test_data, test_data['price'])
print rss_model3_test

2.26568089093e+14
2.24368799994e+14
1.30667348428e+17


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [36]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [ ]:
# create the remaining 3 features in both TEST and TRAIN data



* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)


In [ ]:
# Examine/extract each model's coefficients:


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.

In [37]:
sales = graphlab.SFrame('kc_house_data.gl/')

In [488]:
import numpy as np
from graphlab import SFrame

In [1432]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1.0 # add a constant column to an SFrame
    constant = data_sframe['constant']
    # prepend variable 'constant' to the features list
    #print features
    #print constant
    features = SFrame({'constant':constant, 'features':features})
    features = features.unpack('features')
    #print features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe = features[:]
    #print features_sframe
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    #print features_matrix
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray = data_sframe['price']
    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [1433]:
def predict_outcome(features_matrix, weights):
    weights = np.array(weights)
    #print weights
    #print features_matrix
    predictions = np.dot(features_matrix, weights) # note that the weights are on the right
    return(predictions)
    #print predictions

In [1434]:
def feature_derivative(errors, feature):
    #If we have a the values of a single input feature in an array ‘feature’ and the prediction ‘errors’ (predictions - output) 
    #then the derivative of the regression cost function with respect to the weight of ‘feature’ is just twice the dot product 
    #between ‘feature’ and ‘errors’. Write a function that accepts a ‘feature’ array and ‘error’ array and returns the 
    #‘derivative’ (a single number). e.g. in python:
    errors = predictions - output_array
    derivative = 2 * np.dot(feature, errors)
    return(derivative)

In [1435]:
def regression_gradient_descent(features_matrix, output, initial_weights, step_size, tolerance):
    converged = False
    weights = np.array(initial_weights)
    #print features_matrix
    #print 'Length of features_matrix',len(features_matrix)
    #print weights
    #print 'Length of weights',len(weights)
    while not converged:
        # compute the predictions based on feature_matrix and weights:
        predictions = np.dot(features_matrix, weights)
        # compute the errors as predictions - output:
        #print predictions
        #print 'Length of preditions',len(predictions)
        errors = predictions - output
        #print errors
        #print 'Length of errors',len(errors)
        gradient_sum_squares = 0 # initialize the gradient
        # while not converged, update each weight individually:
        for i in range(len(weights)):
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            feature = features_matrix[:, i]
            # compute the derivative for weight[i]:
            derivative = 2 * np.dot(feature.T, errors)
            #print derivative
            # add the squared derivative to the gradient magnitude
            gradient_sum_squares = gradient_sum_squares + (derivative**2)
            # update the weight based on step size and derivative:
            weights[i] = weights[i] - np.dot(step_size, derivative)
        gradient_magnitude = np.sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [1436]:
train_data,test_data = sales.random_split(.8,seed=0)

In [1437]:
simple_features = train_data['sqft_living']
#print simple_features
my_output = train_data['price']
#print my_output
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

TypeError: Only SArray of dict/list/array type supports unpack

In [1438]:
simple_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
print simple_weights

ValueError: shapes (17384,1) and (3,) not aligned: 1 (dim 1) != 3 (dim 0)

In [1439]:
simple_features = test_data['sqft_living']
#print simple_features
test_output = test_data['price']
#print my_output
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, test_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

TypeError: Only SArray of dict/list/array type supports unpack

In [1440]:
test_predict = predict_outcome(test_simple_feature_matrix, simple_weights)

ValueError: shapes (4229,1) and (2,) not aligned: 1 (dim 1) != 2 (dim 0)

In [1441]:
print test_predict[0]

356134.443171


In [1442]:
test_data['price'].head()

dtype: float
Rows: 10
[310000.0, 650000.0, 233000.0, 580500.0, 535000.0, 605000.0, 775000.0, 292500.0, 289000.0, 571000.0]

In [1443]:
residuals = test_predict - test_data['price']
RSS = residuals**2
print RSS.sum()

2.75400047593e+14


In [1448]:
model2_features = train_data['sqft_living','sqft_living15']
my_output = train_data['price']
(feature2_matrix, output) = get_numpy_data(train_data, model2_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [1449]:
model2_weights = regression_gradient_descent(feature2_matrix, output, initial_weights, step_size, tolerance)
print model2_weights

[ -9.99999688e+04   2.45072603e+02   6.52795277e+01]


In [1452]:
model2_features = test_data['sqft_living','sqft_living15']
my_output = test_data['price']
(feature2_matrix, output) = get_numpy_data(test_data, model2_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [1453]:
test2_predict = predict_outcome(feature2_matrix, model2_weights)

In [1454]:
print test2_predict[0]

366651.412037


In [1455]:
residuals = test2_predict - test_data['price']
RSS = residuals**2
print RSS.sum()

2.70263446465e+14
